## Import Libraries

In [1]:
from pyspark.sql.functions import hour,minute,second,col,avg,when
from pyspark.sql import SQLContext
import pyspark.sql.functions as sqlFun
import datetime
sqlContext = SQLContext(sc)

## Read csv file from hdfs server

In [2]:
df_log = sqlContext.read.csv("hdfs:///user/spark-file/output/all",header = True, inferSchema = True)
df_log.show(10)

+--------------------+-------------------+-------------------+-------------------+-------------------+
|           user_name|         start_time|           end_time|          idle_time|       working_hour|
+--------------------+-------------------+-------------------+-------------------+-------------------+
|prathameshsalap@g...|2019-10-21 08:35:01|2019-10-21 19:25:02|2019-10-21 01:50:00|2019-10-21 09:00:01|
|vaishusawant143@g...|2019-10-21 08:35:01|2019-10-21 19:25:02|2019-10-21 01:35:00|2019-10-21 09:15:01|
|     you@example.com|2019-10-21 08:35:01|2019-10-21 19:25:01|2019-10-21 01:55:00|2019-10-21 08:55:00|
|  shelkeva@gmail.com|2019-10-21 08:50:02|2019-10-21 19:25:02|2019-10-21 01:00:00|2019-10-21 09:35:00|
|ashutoshrit64@gma...|2019-10-21 08:50:01|2019-10-21 19:25:02|2019-10-21 00:30:00|2019-10-21 10:05:01|
|akshaypatwari24@g...|2019-10-21 09:00:01|2019-10-21 19:15:01|2019-10-21 01:30:00|2019-10-21 08:45:00|
|sushantpatwari123...|2019-10-21 09:05:01|2019-10-21 19:00:01|2019-10-21 

In [3]:
print('Rows : ',df_log.count(), 'Columns : ',len(df_log.columns))    # total no of row and column

Rows :  307 Columns :  5


In [12]:
df_log = df_log.drop('working_hour', 'idle_time', 'end_time')

In [13]:
df_start = df_log.withColumn('start_time', sqlFun.from_unixtime(sqlFun.unix_timestamp(df_log.start_time),'hh:mm:ss'))
df_start.show(5)

+--------------------+----------+
|           user_name|start_time|
+--------------------+----------+
|prathameshsalap@g...|  08:35:01|
|vaishusawant143@g...|  08:35:01|
|     you@example.com|  08:35:01|
|  shelkeva@gmail.com|  08:50:02|
|ashutoshrit64@gma...|  08:50:01|
+--------------------+----------+
only showing top 5 rows



In [14]:
late_commers = df_start.filter(sqlFun.col('start_time') > '10:00:00')
late_commers.show(5)

+--------------------+----------+
|           user_name|start_time|
+--------------------+----------+
|vaibhavkhadke@gma...|  10:00:01|
|vijaykumarbhavanu...|  10:00:01|
|priyankagandhi250...|  10:00:02|
|aheteshams007@gma...|  10:00:01|
|hakepratiksha55@g...|  10:15:01|
+--------------------+----------+
only showing top 5 rows



In [15]:
df_avg_hours = late_commers.withColumn('avg_hour', (hour(late_commers['start_time'])*3600 + minute(late_commers['start_time'])*60 + second(late_commers['start_time']))/3600)
df_avg_hours.show(5,truncate=50)

+----------------------------+----------+------------------+
|                   user_name|start_time|          avg_hour|
+----------------------------+----------+------------------+
|     vaibhavkhadke@gmail.com|  10:00:01|10.000277777777777|
|      vijaykumarbhavanur.com|  10:00:01|10.000277777777777|
|priyankagandhi2504@gmail.com|  10:00:02|10.000555555555556|
|     aheteshams007@gmail.com|  10:00:01|10.000277777777777|
|   hakepratiksha55@gmail.com|  10:15:01|10.250277777777777|
+----------------------------+----------+------------------+
only showing top 5 rows



In [16]:
#calculating average hours
total_avg_late_hour = df_avg_hours.select(avg('avg_hour'))
total_avg_late_hour.show()

+------------------+
|     avg(avg_hour)|
+------------------+
|10.699375687568766|
+------------------+



In [17]:
avg_count = df_avg_hours.filter(sqlFun.col("avg_hour") > 10.699375687568766)
avg_count.count()

29

In [18]:
df_avg_hours.filter(df_avg_hours['avg_hour'] > 10.699375687568766).select('user_name').show(5,truncate=50)

+-------------------------+
|                user_name|
+-------------------------+
|nitishtech.info@gmail.com|
|      grmule018@gmail.com|
|  venkatesh.er2@gmail.com|
| salinabodale73@gmail.com|
|  polelaxman001@gmail.com|
+-------------------------+
only showing top 5 rows

